In [1]:
from astropy.table import Table, vstack, unique, SortedArray
import glob
import numpy
import time
from datetime import date, timedelta, datetime
import psycopg2
import sqlite3
import pandas as pd
from functools import reduce
from desispec.io import read_spectra, write_spectra
from desispec.spectra import Spectra
from desispec.coaddition import coadd

import matplotlib.pyplot as plt
import os
from desiutil.log import get_logger, DEBUG

In [5]:
#get array of yyyymmdds to loop through
#getUnprocessedDates.py
def getUnprocessedDates():
    start_time = time.time()
    # Connect to desi.db with POSTGRES to get latest observed and unprocessed yyyymmdd
    f = open('/global/cfs/cdirs/desi/science/td/secrets/desi_pg.txt') #what is more valid?
    file = f.read()
    db_name, db_user, db_pwd, db_host = file.split()
    conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_pwd, host=db_host)
    cur = conn.cursor()
    cur.execute("""SELECT DISTINCT yyyymmdd from fibermap_daily WHERE yyyymmdd > 20210604""") #most recent, remove in future
    desi_arr = cur.fetchall()
    cur.close()
    conn.close()
    
    # Open to transients_search.db for latest processed yyyymmdd to do comparison with unprocessed
    filename_conn = "/global/cfs/cdirs/desi/science/td/daily-search/transients_search.db"
    conn = sqlite3.connect(filename_conn)
    trans_arr = conn.execute("""SELECT DISTINCT yyyymmdd from unprocessed_exposures""").fetchall()
    conn.close()

    # Compare yyyymmdd from fibermap_daily with unprocessed_exposures, retaining those in fibermap_daily and not in unprocessed_daily
    night_arr = numpy.setdiff1d(desi_arr, trans_arr)

    print('len(night_arr): ' + str(len(night_arr)))
    print("--- get unprocessed dates took:  %s seconds ---" % (time.time() - start_time))
    
    return(night_arr) 

In [48]:
def getMatchedTileid(yyyymmdd):
    
    # read in and store in one place all the fibermap information in the spectra files
    dats=[]
    for filename in glob.glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{yyyymmdd}/spectra-*.fits"):
        t = Table.read(filename, format='fits',hdu=1, memmap=True) 
        ##### DAVE'S ADDITION ##############
        targetcols = [i for i in t.colnames if i[-7:] =='_TARGET']
        nonzerocheck = [True in k for k in [[j != 0 for j in t[targetcols][i]] for i in range(len(t))]]
        #a really ugly line, basically generates a list of bools, 
        #True if there is at least one nonzero element in all columns ending in _TARGET
        t.remove_rows([i for i, val in enumerate(nonzerocheck) if not val])
        #This gets the index of all False values from the previous list and removes those rows
        t = t['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC','FIBERSTATUS','NIGHT']
        ######## END DAVE'S ADDITION ############
        t=t[numpy.logical_and(t['OBJTYPE']=='TGT', t['FIBERSTATUS']==0)]
        dats.append(t)
    dats=vstack(dats, join_type='inner',metadata_conflicts='silent')

    # group all the observations by TARGET_RA and TARGET_DEC
    # note that this is more reliable than grouping by TARGETID as TARGETID is NOT a unique identifier of RA and DEC
    dats_group = dats.group_by(['TARGET_RA','TARGET_DEC'])
    
    # for each group make a list of all targetid's with that ra/dec
    group_tid=[]
    group_night=[]
    # for each group make a tuple containing all tileid/petal combinations that have that ra/dec
    group_tp=[]    
    for g in dats_group.groups:
        group_tid.append(numpy.unique(g['TARGETID'].data))
        group_night.append(numpy.unique(g['NIGHT'].data))
#     for g in dats_group.groups:
        gu = unique(g,keys=['TILEID'])
        dum=[]
        for a,b in zip(gu['TILEID'],gu['PETAL_LOC']):
            dum.append((a,b))
        group_tp.append(tuple(dum))

    # compress things down to the unique tile/petal combinations
    ans = list(set(group_tp))
    
    # union sets that have intersecting tile/petal combinations
    for i in range(len(ans)-1,0,-1):
        for j in range(i-1,-1,-1):
            if len(set(ans[i]) & set(ans[j])) !=0:
                ans[j]=ans[i]+ans[j]
                del ans[i]
                break

    # unique of each set
    for i in range(len(ans)):
        ans[i]=list(set(ans[i]))
        
    tile_petal=ans
        
    # for each set the TARGET_RA TARGET_DEC associated
        
    return(tile_petal, group_tid, group_tp, group_night)

In [49]:
yyyymmdd = 20210606
tile_petal,group_tid,group_tp, group_night = getMatchedTileid(yyyymmdd)

500
381
500
443
500
408
500
454
500
435
500
409
500
377
500
443
500
434
500
415
500
377
500
409
500
415
500
408
500
381
500
454
500
434
500
435
500
445
500
442
500
409
500
377
500
443
500
381
500
454
500
415
500
434
500
435
500
442
500
408
500
381
500
443
500
409
500
443
500
434
500
454
500
435
500
408
500
415
500
377
1000
830
1000
886
1000
908
1000
816
1000
754
1000
890
1000
870
1000
818
1000
868
1000
762
500
442
500
445
500
434
500
408
500


KeyboardInterrupt: 

In [25]:
len(group_tid)

97629

In [42]:
yyyymmdd=20210606
tile_petal,group_tid,group_tp=getMatchedTileid(yyyymmdd)

In [57]:
# tile_petal contain subsets of tile/petals whose RA/DEC targets are not contained in other tile/petals from that night
for tps in tile_petal:
    # cache spectra to minimize IO
    spectra_list=dict()
    target_list=[]
    for tp in tps:
        filename =  f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{tp[0]}/{yyyymmdd}/spectra-{tp[1]}-{tp[0]}-thru{yyyymmdd}.fits"
        spectra_list[(tp[0],tp[1])]=read_spectra(filename)
    
    # loop over all unique RA/DEC pairs from that night
    for tid, tp in zip(group_tid,group_tp):
        # if this RA/DEC is not in thie tile_petal combination than skip
        if tp[0] not in tps:
            continue

        # Proceed with a subtraction for this object
        
        # The coadds of the new and reference are constructed from all spectra with all targetid's in tid and all
        # tile/petal combinations in tp, which are cached above
        print(tid, tp)
        wefew


[(1239, 6)]
INFO:spectra.py:282:read_spectra: iotime 1.013 sec to read spectra-6-1239-thru20210606.fits at 2021-08-13T17:03:41.816200
[39628512646138870] ((1239, 6),)


NameError: name 'wefew' is not defined

In [54]:
spectra_list

{(1239, 6): <desispec.spectra.Spectra at 0x2aaae3a12250>}

In [7]:
night_arr = getUnprocessedDates()
print(night_arr)

len(night_arr): 20
--- get unprocessed dates took:  8.81651496887207 seconds ---
[20210606 20210607 20210614 20210615 20210616 20210617 20210618 20210619
 20210620 20210621 20210626 20210627 20210628 20210629 20210704 20210705
 20210706 20210707 20210708 20210709]


In [ ]:
### for debugging purposes remove 'for a in night_arr:'-loop, assign yyyymmdd = 20210606 or 20210406 for example


for a in night_arr:   
    # read in and store in one place all the fibermap information in the spectra files
    yyyymmdd = a
    dats=[]
    for filename in glob.glob(f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/*/{yyyymmdd}/spectra-*.fits"):
        t = Table.read(filename, format='fits',hdu=1, memmap=True)[['TARGETID','TARGET_RA','TARGET_DEC','TILEID','OBJTYPE','PETAL_LOC', 'NIGHT']]
        t=t[t['OBJTYPE']=='TGT']
        dats.append(t)
    dats=vstack(dats, join_type='inner',metadata_conflicts='silent')
    # group all the observations by TARGET_RA and TARGET_DEC
    # note that this is more reliable than grouping by TARGETID as TARGETID is NOT a unique identifier of RA and DEC
    dats_group = dats.group_by(['TARGET_RA','TARGET_DEC'])
    
    # for each group make a tuple containing all tileid/petal combinations that have that ra/dec
    tile_petal=[]
    # for each group, determine the unique nights that occur
    night=[]
    for g in dats_group.groups:
        
        gu = unique(g, keys=['TILEID'])
        
        dum=[]
        for a,b,c in zip(gu['TILEID'],gu['PETAL_LOC'],gu['TARGETID']):
            dum.append((a,b,c))
        tile_petal.append(tuple(dum))


        du = unique(g, keys=['NIGHT'])
        
        for d in zip(du['NIGHT']):
            night.append((d))
        

    # compress things down to the unique tile/petal combinations
    tile_petal = list(set(tile_petal))
    # union sets that have intersecting tile/petal combinations
    for i in range(len(tile_petal)-1,0,-1):
        for j in range(i-1,-1,-1):
            if len(set(tile_petal[i]) & set(tile_petal[j])) !=0:
                tile_petal[j]=tile_petal[i]+tile_petal[j]
                del tile_petal[i]
                break
    # unique of each set
    for i in range(len(tile_petal)):
        tile_petal[i]=list(set(tile_petal[i]))
    
    unique_night = numpy.unique(night)
    
    for i in tile_petal:
        spectra = Spectra()
        dif_spectra = Spectra()
        ref_spectra = Spectra() ### or else NameError at 'if ref_spectra.num_spectra'... in the case of no reference nights
        for j in i:    ### covers case of multiple tile/petal pairs 
            filename =  f"/global/project/projectdirs/desi/spectro/redux/daily/tiles/cumulative/{j[0]}/{yyyymmdd}/spectra-{j[1]}-{j[0]}-thru{yyyymmdd}.fits"
            spectra = read_spectra(filename)
            date = numpy.array(yyyymmdd)
            idx = numpy.in1d(unique_night, date)
            ref_night = unique_night[~idx]

            try:
                cur_spectra = (spectra.select(nights=date, targets = [j[2]]))
                ref_spectra = (spectra.select(nights=ref_night, targets = [j[2]]))
            except:
                pass ### this is the mechanism for eliminating targets that have no reference nights. 

            if ref_spectra.num_spectra() != 0:
            ### in the case of no ref nights, as 'pass' has allowed a reference spectra to be created despite a RuntimeError, this is the check that there is something to difference
                coadd(ref_spectra)
                dif_spectra.update(cur_spectra)
                coadd(dif_spectra)
                for b in dif_spectra.bands:
                    dif_spectra.flux[b][0] = cur_spectra.flux[b][0]-ref_spectra.flux[b][0]
                    ok = numpy.logical_and(dif_spectra.mask[b][0,:] == 0, dif_spectra.ivar[b][0] != 0)
                    plt.plot(cur_spectra.wave[b][ok], cur_spectra.flux[b][0,ok], color = 'red', label = "current spectra")
                    plt.plot(ref_spectra.wave[b][ok],ref_spectra.flux[b][0,ok], color = 'green', label = 'reference spectra')
                    plt.plot(dif_spectra.wave[b][ok],dif_spectra.flux[b][0,ok], color = 'cyan', label = 'differenced spectra')
                    if b == 'b':
                        plt.legend()
                plt.show()
                
                # save each days candidate plots into one pdf
                with PdfPages('diff_candidates(%d).pdf' % yyyymmdd) as pdf:
                    fig = plt.figure()
                    pdf.savefig(fig)

    ### insert garbage collection potentially
            
    
    